In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv")

In [ ]:
df.head()

In [ ]:
df.describe()

No missing values

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

## EDA

In [ ]:
plt.figure(1 , figsize = (15 , 6))
n = 0 
for x in ['Age' , 'Annual Income (k$)' , 'Spending Score (1-100)']:
    n += 1
    plt.subplot(1 , 3 , n)
    sns.distplot(df[x] , bins = 20)
    plt.title('Distplot of {}'.format(x))
plt.show()

In [ ]:
sns.heatmap(df.corr())

## K Means Clustering

In [ ]:
x = df[['Age','Annual Income (k$)','Spending Score (1-100)']]

In [ ]:
wcss = []
score = []
for n in range(2,10):
    kmeans = KMeans(n)
    kmeans.fit(x)
    wcss.append(kmeans.inertia_)
    score.append(metrics.silhouette_score(x,kmeans.labels_))

### Get range from Elbow method and finalize number of clusters (k) using Silhouette Coefficient

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(go.Scatter(x=tuple(range(2,10)), y=wcss,
                    mode='lines+markers',
                    name='lines+markers'))
fig.update_xaxes(title_text="Number of clusters")
fig.update_yaxes(title_text="Within Cluster Sum of Squares")
fig.show()

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(go.Scatter(x=tuple(range(2,10)), y=score,
                    mode='lines+markers',
                    name='lines+markers'))
fig.update_yaxes(title_text="Silhouette Coefficient")
fig.update_xaxes(title_text="Number of clusters")
fig.show()

We can see that the best result is possible at cluster n=6.

In [ ]:
kmeans = KMeans(6)
kmeans.fit(x)

In [ ]:
x_clusters = x.copy()
x_clusters['Cluster'] = kmeans.labels_

## Visualization

In [ ]:
fig, (axes) = plt.subplots(1,2,figsize=(12,5))
sns.scatterplot(x=x_clusters['Age'], y=x_clusters['Spending Score (1-100)'],
                hue=x_clusters['Cluster'],ax=axes[0], palette='Set1', legend='full')
sns.scatterplot(x=x_clusters['Annual Income (k$)'], y=x_clusters['Spending Score (1-100)'],
                hue=x_clusters['Cluster'],ax=axes[1], palette='Set1', legend='full')

In [ ]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(x_clusters, x='Age', y='Spending Score (1-100)', z='Annual Income (k$)',
              color='Cluster')
fig.show()

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE()
palette = sns.color_palette("bright", 6)
X_embedded = tsne.fit_transform(x)
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=kmeans.labels_, legend='full', palette=palette)